In [1]:
from tqdm import tqdm


### Filter ES ticks into a single continous contract

In [3]:
csv = "../../../../Historical Data/ES-Futures-Ticks-20230807-20240806.trades.csv" # input
filtered_csv = "../../../../Historical Data/ES-Futures-Ticks-20230807-20240806.trades.filtered.csv" # output

with open(csv, "r") as f:
    for i in f:
        print("Column headers: \n", "\n".join(
            [str(x) for x in enumerate(i.split(","))]))
        break

Column headers: 
 (0, 'ts_recv')
(1, 'ts_event')
(2, 'rtype')
(3, 'publisher_id')
(4, 'instrument_id')
(5, 'action')
(6, 'side')
(7, 'depth')
(8, 'price')
(9, 'size')
(10, 'flags')
(11, 'ts_in_delta')
(12, 'sequence')
(13, 'symbol\n')


In [4]:
"""
We want ts_event as time, price, size as volume, symbol
So we want column numbers 1, 8, 9, 13
The symbols we want are ESU3, ESZ3, ESH4, ESM4, ESU4
"""
data = {"time" : [], "price" : [], "volume" : [], "symbol" : []}
with open(csv, "r") as f:
    for i, line in enumerate(tqdm(f, total=106615135)):
        if i == 0:
            continue
        line = line.strip().split(",")
        if line[13] not in ["ESU3", "ESZ3", "ESH4", "ESM4", "ESU4"]:
            continue
        data["time"].append(line[1])
        data["price"].append(line[8])
        data["volume"].append(line[9])
        data["symbol"].append(line[13])

  0%|          | 0/106615135 [00:00<?, ?it/s]

100%|██████████| 106615135/106615135 [01:10<00:00, 1503477.46it/s]


In [5]:
"""
We start to see some trades of ESZ3 appearing during the ESU3 contract
Let's further filter the data so that each contract reaches its end date
prior to admitting prices from the next contract
To do this, we need to identify the end index of each contract
"""
esu3_end = 0
esz3_end = 0
esh4_end = 0
esm4_end = 0

for i, s in enumerate(tqdm(data["symbol"])):
    if s == "ESU3":
        esu3_end = i
    elif s == "ESZ3":
        esz3_end = i
    elif s == "ESH4":
        esh4_end = i
    elif s == "ESM4":
        esm4_end = i

print(esu3_end, esz3_end, esh4_end, esm4_end)

  0%|          | 0/105736599 [00:00<?, ?it/s]

100%|██████████| 105736599/105736599 [01:09<00:00, 1525311.72it/s]

12304220 41704811 64404499 89984159


In [6]:
print(f"""
ESU3: {data["time"][0]} - {data["time"][esu3_end]}
ESZ3: {data["time"][esu3_end]} - {data["time"][esz3_end]}
ESH4: {data["time"][esz3_end]} - {data["time"][esh4_end]}
ESM4: {data["time"][esh4_end]} - {data["time"][esm4_end]}
      """.strip())

ESU3: 2023-08-07T00:00:00.039623665Z - 2023-09-15T13:29:59.452410495Z
ESZ3: 2023-09-15T13:29:59.452410495Z - 2023-12-15T14:29:59.410358005Z
ESH4: 2023-12-15T14:29:59.410358005Z - 2024-03-15T13:29:59.847946481Z
ESM4: 2024-03-15T13:29:59.847946481Z - 2024-06-21T13:29:59.522321675Z


In [7]:
filtered_data = {"time" : [], "price" : [], "volume" : [], "symbol" : []}
for i in tqdm(range(len(data["symbol"]))):
    if (i <= esu3_end and data["symbol"][i] == "ESU3") or \
       (i > esu3_end and i <= esz3_end and data["symbol"][i] == "ESZ3") or \
       (i > esz3_end and i <= esh4_end and data["symbol"][i] == "ESH4") or \
       (i > esh4_end and i <= esm4_end and data["symbol"][i] == "ESM4") or \
       (i > esm4_end and data["symbol"][i] == "ESU4"):
        filtered_data["time"].append(data["time"][i])
        filtered_data["price"].append(data["price"][i])
        filtered_data["volume"].append(data["volume"][i])
        filtered_data["symbol"].append(data["symbol"][i])


100%|██████████| 105736599/105736599 [01:20<00:00, 1309709.43it/s]


In [8]:
len(data["symbol"]), len(filtered_data["symbol"])

(105736599, 82971690)

In [9]:
with open (filtered_csv, "w") as f:
    f.write("time,price,volume,symbol\n")
    for i in tqdm(range(len(filtered_data["symbol"]))):
        f.write(f"{filtered_data['time'][i]},{filtered_data['price'][i]},{filtered_data['volume'][i]},{filtered_data['symbol'][i]}\n")

100%|██████████| 82971690/82971690 [01:26<00:00, 958235.74it/s] 


### Once the above data is filtered, we can read it in on subsequent runs. 

In [1]:
from tqdm import tqdm
import pandas as pd
import sys; sys.path.append("..")
from bars.imbalance_bars import tick_imbalance_bars, volume_imbalance_bars
# from utils.visualizations.plot_bars_on_prices import plot_bars_on_prices
import plotly.graph_objects as go

In [2]:
filtered_csv = "../../../../Historical Data/ES-Futures-Ticks-20230807-20240806.trades.filtered.csv" # now input

data = {"time" : [], "price" : [], "volume" : [], "symbol" : []}
with open(filtered_csv, "r") as f:
    for i, line in enumerate(tqdm(f, total=82971690)):
        if i == 0:
            continue
        line = line.strip().split(",")
        data["time"].append(line[0])
        data["price"].append(float(line[1]))
        data["volume"].append(float(line[2]))
        data["symbol"].append(line[3])

82971691it [00:46, 1772621.64it/s]                              


In [4]:
bars = volume_imbalance_bars(data, alpha=0.001, et_init=500, verbose=True)

  0%|          | 189866/82971689 [00:00<01:25, 966945.10it/s]

Making a new bar from 0 to 1180
Making a new bar from 1180 to 12307
Making a new bar from 12307 to 24594
Making a new bar from 24594 to 54383
Making a new bar from 54383 to 56698
Making a new bar from 56698 to 59482
Making a new bar from 59482 to 81001
Making a new bar from 81001 to 95110


  1%|          | 503808/82971689 [00:00<00:59, 1375885.15it/s]

Making a new bar from 95110 to 224197
Making a new bar from 224197 to 240711
Making a new bar from 240711 to 315876
Making a new bar from 315876 to 319891
Making a new bar from 319891 to 352007
Making a new bar from 352007 to 368415
Making a new bar from 368415 to 518958
Making a new bar from 518958 to 521209
Making a new bar from 521209 to 521245
Making a new bar from 521245 to 521246
Making a new bar from 521246 to 521249
Making a new bar from 521249 to 521252
Making a new bar from 521252 to 521264
Making a new bar from 521264 to 521265
Making a new bar from 521265 to 521269
Making a new bar from 521269 to 521270
Making a new bar from 521270 to 521278
Making a new bar from 521278 to 521303
Making a new bar from 521303 to 521400
Making a new bar from 521400 to 521547
Making a new bar from 521547 to 521925
Making a new bar from 521925 to 521926
Making a new bar from 521926 to 521929
Making a new bar from 521929 to 521952
Making a new bar from 521952 to 521960
Making a new bar from 5219

  1%|          | 821756/82971689 [00:00<00:59, 1381567.20it/s]

...

  5%|▍         | 3946298/82971689 [00:02<01:01, 1292480.03it/s]

Making a new bar from 3428676 to 3818177


 15%|█▍        | 12120041/82971689 [00:08<00:43, 1618924.05it/s]

Making a new bar from 3818177 to 12214178


100%|██████████| 82971689/82971689 [01:01<00:00, 1342324.69it/s]


In [5]:
len(bars['start_idx'])

1570

In [6]:
# use plotly to plot the bars as candles
fig = go.Figure(data=[go.Candlestick(x=[i for i in range(len(bars['start_idx']))],
                open=bars['open'],
                high=bars['high'],
                low=bars['low'],
                close=bars['close'])])

fig.show()